# imports

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from SEAM.utils import *
import SEAM
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cv2
from sklearn.decomposition import PCA

from scipy.spatial.distance import *

In [ ]:
from scipy.spatial import *

# settings

In [ ]:
sns.set_style('white')
# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 2}

# matplotlib.rc('font', **font)
# matplotlib.rcParams.update({'font.size': 2})

import matplotlib.pyplot as plt

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title




tiff_path = '/home/yzy/PUBDT/st/MIBI/BC/tiff/'
seg_path = '/home/yzy/PUBDT/st/MIBI/BC/segmentation_mask/'
h5ad_path = '/home/yzy/PUBDT/st/MIBI/BC/h5ad/'

var_list = ['Au','Background','Beta catenin','Ca','CD11b','CD11c','CD138','CD16',
           'CD20','CD209','CD3','CD31','CD4','CD45','CD45RO','CD56','CD63','CD68',
           'CD8','dsDNA','EGFR','Fe','FoxP3','H3K27Me3','H3K9ac','HLA-DR','HLA_Class_1',
           'IDO','Keratin17','Keratin6','Ki67','Lag3','MPO','Na','P','p53','Pan-Keratin',
           'PD-L1','PD1','phospho-S6','Si','SMA','Ta','Vimentin']

protein_list = ['Beta catenin','CD11b','CD11c','CD138','CD16',
           'CD20','CD209','CD3','CD31','CD4','CD45','CD45RO','CD56','CD63','CD68',
           'CD8','dsDNA','EGFR','FoxP3','H3K27Me3','H3K9ac','HLA-DR','HLA_Class_1',
           'IDO','Keratin17','Keratin6','Ki67','Lag3','MPO','p53','Pan-Keratin',
           'PD-L1','PD1','phospho-S6','SMA','Vimentin']


image_size = 2048
# SEAM.settings.IMG_SZ1 = 2048
# SEAM.settings.IMG_SZ2 = 2048

# functions`

In [ ]:
# distmat = adata_sc_me.obsm['X_ME_EMD_mat']

# gt_obs = 'CT_gt'
# adata_use = adata_sc_me
# n_splits = 10
from sklearn.model_selection import *
from sklearn.neighbors import *
def compare_KNN(distmat,gt_obs,adata_use,n_splits):
    kf = StratifiedKFold(n_splits=n_splits,shuffle=True)
    knn = KNeighborsClassifier(metric='precomputed')
    cur_rst=cross_val_score(knn, distmat, adata_use.obs[gt_obs], scoring='accuracy',cv=kf,verbose=0)
#     np.mean(cur_rst)
    return cur_rst

In [ ]:
def prepare_adata_for_SIMSID(img_mat,segment_mat,var_names):

    # prepare cell_pos
    cell_pos = np.where(segment_mat.flatten()!=0)[0]

    # prepare cell_idx
    cell_idx = np.zeros_like(cell_pos)
    cell_idx_raw = segment_mat[segment_mat!=0]
    cell_idx_raw_unique = np.unique(cell_idx_raw)
    cell_idx_raw_unique_sort = np.sort(cell_idx_raw_unique)
    for cir in range(len(cell_idx_raw)):
        cur_true_idx = np.where(cell_idx_raw_unique_sort==cell_idx_raw[cir])[0][0]
        cell_idx[cir]=cur_true_idx

    cell_idx+=1

    # prepare train_x
    train_x_raw = img_mat.reshape(img_mat.shape[0],-1).transpose()
    train_x = train_x_raw[cell_pos,:]

    # prepare IMS
    IMS = img_mat.reshape(img_mat.shape[0],-1).transpose()


    # prepare sc_mat
    sc_mat = np.zeros(shape=(cell_idx.max(),img_mat.shape[0]))
    for i in range(sc_mat.shape[0]):
        cur_pixels = train_x[cell_idx==i+1,:]
        cur_sum = np.mean(cur_pixels,axis=0)
        sc_mat[i,:] = cur_sum

    adata_sc = ad.AnnData(X=sc_mat)
    adata_sc.var_names = var_names
    adata_sc.uns['cell_pos'] = cell_pos
    adata_sc.uns['cell_idx'] = cell_idx
    adata_sc.uns['train_x'] = train_x
    adata_sc.uns['IMS'] = IMS
    # adata_sc.var_names = 
    return adata_sc

### utils

In [ ]:
def generate_spatialmat(sz=256):
    rst = []
    for i in range(sz):
        for j in range(sz):
            rst.append([i,j])
    return np.array(rst)

### get_ground_distance

In [ ]:
import pyemd
from scipy.spatial.distance import *
def get_ground_distance(adata,method='paga'): 
    
    if method == 'paga_graph':
        adj_mat_use = adata.uns['paga']['connectivities'].toarray()
#         adj_mat_use[adj_mat_use==0] = -np.inf
#         adj_mat_use = 1-adj_mat_use
        adj_mat_use = 1/adj_mat_use

        np.fill_diagonal(adj_mat_use,0)
        G = nx.from_numpy_matrix(adj_mat_use)

        len_path = dict(nx.all_pairs_dijkstra_path_length(G))
        ground_distance_mat = np.inf*np.ones(shape=(len(len_path),len(len_path)))

        for i in range(ground_distance_mat.shape[0]):
            for j in range(ground_distance_mat.shape[1]):
                ground_distance_mat[i,j] = len_path[i][j]
        np.fill_diagonal(ground_distance_mat,0)
    elif method =='paga':
        adj_mat_use = adata.uns['paga']['connectivities'].toarray()
#         adj_mat_use[adj_mat_use==0] = -np.inf
        adj_mat_use = 1-adj_mat_use
        np.fill_diagonal(adj_mat_use,0)
        ground_distance_mat = adj_mat_use
    elif method =='euclidean':   #lower bound of MED(approximation)
        ground_distance_mat = squareform(pdist(adata.uns['paga']['pos']))
    elif method == 'uniform':
        sz = adata.uns['paga']['pos'].shape[0]
        ground_distance_mat = np.ones(shape=(sz,sz))
        np.fill_diagonal(ground_distance_mat,0)
        
    adata.uns['GD'] = ground_distance_mat
    return ground_distance_mat

### MED_phEMD

In [ ]:
def MED_phEMD(adata,  
              GD_method='euclidean',
              MED_knn=30,
              CT_obs='clusters',
              ifspatialplot=True,
              OT_method='pyemd',
              ME_precompyted=False,
              GD_precomputed=False
             ):
    
    
    # calculate ground distance, paga requried
    if not GD_precomputed:
        ground_distance_mat = get_ground_distance(adata,method=GD_method)
    else:
        ground_distance_mat = adata.uns['GD']
    # get ME histogram
    if not ME_precompyted:
        MED(adata,CT_obs,MED_knn)
    ME_mat = adata.obsm['ME']


    # calculate EMD matrix
    import time
    ME_mat = adata.obsm['ME']
    ME_dist_EMD = np.zeros(shape=(ME_mat.shape[0],ME_mat.shape[0]))
    time_cost = 0
    time_start = 0
    time_end = 0
    for i in range(ME_mat.shape[0]):
        if i%100==0:
            time_end = time.time()
            time_cost = time_end-time_start
            print(i,time_cost)
            time_start = time.time()

        for j in range(i,ME_mat.shape[0]):
            first_histogram = ME_mat[i,:]
            second_histogram = ME_mat[j,:]
            first_histogram = first_histogram/np.sum(first_histogram)
            second_histogram = second_histogram/np.sum(second_histogram)
            if OT_method=='pyemd':
                cur_dist = pyemd.emd(first_histogram,second_histogram,ground_distance_mat)
    #         print(i,j,cur_dist)
            elif OT_method=='POT':
                first_histogram = first_histogram/np.sum(first_histogram)
                second_histogram = second_histogram/np.sum(second_histogram)
                cur_dist = ot.emd2(first_histogram,second_histogram,ground_distance_mat)
            ME_dist_EMD[i,j] = cur_dist
            ME_dist_EMD[j,i] = cur_dist
    adata.obsm['X_ME_EMD_mat'] = ME_dist_EMD.copy()
    
    return adata

### tiff loading

In [ ]:
from PIL import Image
import numpy as np

def read_tiff(path):
    """
    path - Path to the multipage-tiff file
    """
    img = Image.open(path)
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img))
    return np.array(images)

In [ ]:
def load_scMEP(Pn,iflog=True):
    print('Point{0} start!'.format(Pn))
#     Pn = 21
    base_path = '/home/yzy/PUBDT/st/MIBI/colorectal_carcinoma/'
    tiff_path = base_path+'scMEP_MIBI_colon_images/'
    segmentation_file = base_path+'segmentation/Point{0}_Label_Map.tiff'

    sc_file = base_path+'scMEP_MIBI_singlecell.csv'
    sc_file_h5ad = base_path+'h5ad/MIBI.h5ad'
    # 1~913
    print('loading processed single cell data...')
    a_sc = sc.read_h5ad(sc_file_h5ad)
    a_sc_test = a_sc[a_sc.obs['point']==str(Pn)].copy()
    del a_sc


    if Pn>=1 and Pn<=20:
        tiff_file = tiff_path+'90de_tumor/Point{0}/'
    elif Pn>=21 and Pn<=40:
        tiff_file = tiff_path+'21d7_tumor/Point{0}/'
    elif Pn>=41 and Pn <=44:
        tiff_file = tiff_path+'d3d3_healthy/Point{0}/'
    elif Pn>=45 and Pn<=58:
        tiff_file = tiff_path+'99c0_healthy/Point{0}/'
        
    print('loading raw tiff data...')

    tiff_mat = []
    protein_list = []
    tiff_file = tiff_file.format(Pn)
    for file in os.listdir(tiff_file):
        tiff = read_tiff(tiff_file+file)
        tiff_mat.append(tiff[0,:])
        cur_protein = file.split('_')[1].split('.')[0]
        protein_list.append(cur_protein)
    segment_mat = read_tiff(segmentation_file.format(Pn))[0,:]
    tiff_mat = np.array(tiff_mat)
    if iflog:
        tiff_mat=np.log1p(tiff_mat)
    

#     start: filter segment_mat according to cell size
    cell_size_range = (a_sc_test.obs['cell_size'].min(),a_sc_test.obs['cell_size'].max())
    for i in range(segment_mat.max()):
        cur_cell_idx = i+1
        cur_cell_pos = np.where(segment_mat==cur_cell_idx)
        if (len(cur_cell_pos[0])>=cell_size_range[0] and len(cur_cell_pos[0])<=cell_size_range[1]):
            continue
        segment_mat[cur_cell_pos[0],cur_cell_pos[1]]=0
    old_cell_idx_unique = np.unique(segment_mat)
    new_cell_idx_unique = np.arange(old_cell_idx_unique.shape[0])
    cell_idx_dict = dict(zip(old_cell_idx_unique,new_cell_idx_unique))

    segment_mat_new = segment_mat.copy()
    for i in range(segment_mat.shape[0]):
        for j in range(segment_mat.shape[1]):
            segment_mat_new[i,j] = cell_idx_dict[segment_mat[i,j]]
    segment_mat = segment_mat_new
#     end: filter segment_mat according to cell size
    
    
    print('spatial matching...')
    segment_mat_rot = np.rot90(segment_mat,3)
    segment_mat_flip = np.flip(segment_mat_rot,axis=1)

    tiff_mat_rot = [np.rot90(t,3) for t in tiff_mat]
    tiff_mat_flip = [np.flip(t,axis=1) for t in tiff_mat_rot]
    tiff_mat_flip = np.array(tiff_mat_flip)
    
    print('data formatting...')
    a_pixel = prepare_adata_for_SIMSID(tiff_mat_flip,segment_mat_flip,protein_list)
    SEAM.tl.Cut(a_pixel)

    cell_size_range = (a_sc_test.obs['cell_size'].min(),a_sc_test.obs['cell_size'].max())
    a_pixel_filtered = a_pixel[np.logical_and(a_pixel.obs['cell_size']>=cell_size_range[0],a_pixel.obs['cell_size']<=cell_size_range[1])]
    print('label matching...')

    spatial_sc = a_sc_test.obsm['spatial']
    spatial_pixel = a_pixel_filtered.obsm['spatial']
    cross_dist_mat = distance_matrix(spatial_pixel,spatial_sc)

    cell_type_source = np.array(a_sc_test.obs['Cluster'])
    match_mat = np.argmin(cross_dist_mat,axis=1)
    cell_match = np.argmin(cross_dist_mat,axis=0)
    cell_type_target = cell_type_source[match_mat]
#     a_pixel_match = a_pixel_filtered[match_mat,:]
    a_pixel_filtered.obs['Cluster'] = cell_type_target
    print('Point{0} done!'.format(Pn))
    return a_pixel_filtered,a_sc_test,cell_match

# data loading steps

In [ ]:
Pn=1
a_pixel,a_sc,cell_match = load_scMEP(Pn)

In [ ]:
cell_match

In [ ]:

sc.pl.embedding(a_pixel,basis='spatial',color=['Cluster'],s=a_pixel.obs['cell_size']/10)
sc.pl.embedding(a_sc,basis='spatial',color=['Cluster'],s=a_sc.obs['cell_size']/10)

In [ ]:
adata = a_pixel

In [ ]:
n_pixels_all = int(adata.uns['IMS'].shape[0])
n_pixels_side = int(np.sqrt(n_pixels_all))

In [ ]:

mask = np.zeros(shape=(n_pixels_all,))
mask[adata.uns['cell_pos']-1] = 1
plt.imshow(mask.reshape(n_pixels_side,n_pixels_side))


cellidx = np.zeros(shape=(n_pixels_all,))
cellidx[adata.uns['cell_pos']-1] = adata.uns['cell_idx']

In [ ]:
adata_pixel_all = ad.AnnData(adata.uns['IMS'])
adata_pixel_all.var_names = adata.var_names
adata_pixel_all.obs['mask'] = mask
adata_pixel_all.obs['cellidx'] = cellidx
adata_pixel_all.obs['cellidx'] = adata_pixel_all.obs['cellidx'].astype('category')

In [ ]:
adata.obs['Cluster'][int(cellidx[i]-1)]

In [ ]:
cell_label = np.zeros_like(cellidx).astype('str')
sc_label = np.array(adata.obs['Cluster'])
for i in range(len(cellidx)):
    if cellidx[i]==0:
        continue
    
    cur_label = sc_label[int(cellidx[i]-1)]
    cell_label[i] = cur_label

In [ ]:
adata_pixel_all.obs['cell_label'] = cell_label.astype('str')
adata_pixel_all.obs['cell_label'] = adata_pixel_all.obs['cell_label'].astype('category')

In [ ]:
adata_pixel_all.obsm['spatial'] = generate_spatialmat(n_pixels_side)

In [ ]:
np.unique(adata_pixel_all.obs['cell_label'],return_counts=1)

In [ ]:
sc.pl.embedding(adata_pixel_all,basis='spatial',color=['mask','cell_label'],size=10)

In [ ]:
adata_pixel_all

In [ ]:
adata_pixel_all
# mask: 0 for background, 1 for cell/nuclei
# cellidx: 0 for background, 1... for cell index
# cell_label: 0 for background, 1... for ground truth cell type

In [ ]:
adata_pixel_all.write_h5ad('/home/yzy/PUBDT/st/IMS/SIMS/cellline/NCTC1469withIdU_Hepa1-6_pixel.h5ad')

# data loading wrap

In [ ]:

def scMEP_h5ad(Pn):
    save_path = '/home/yzy/PUBDT/st/MIBI/colorectal_carcinoma/h5ad/scMEP_pixel_CT_mask_h5ad/'
    
    save_file = save_path+'{0}.h5ad'.format(Pn)
    a_pixel,a_sc,cell_match = load_scMEP(Pn)


    # sc.pl.embedding(a_pixel,basis='spatial',color=['Cluster'],s=a_pixel.obs['cell_size']/10)
    # sc.pl.embedding(a_sc,basis='spatial',color=['Cluster'],s=a_sc.obs['cell_size']/10)

    adata = a_pixel

    n_pixels_all = int(adata.uns['IMS'].shape[0])
    n_pixels_side = int(np.sqrt(n_pixels_all))


    mask = np.zeros(shape=(n_pixels_all,))
    mask[adata.uns['cell_pos']-1] = 1
    # plt.imshow(mask.reshape(n_pixels_side,n_pixels_side))


    cellidx = np.zeros(shape=(n_pixels_all,))
    cellidx[adata.uns['cell_pos']-1] = adata.uns['cell_idx']

    adata_pixel_all = ad.AnnData(adata.uns['IMS'])
    adata_pixel_all.var_names = adata.var_names
    adata_pixel_all.obs['mask'] = mask
    adata_pixel_all.obs['cellidx'] = cellidx
    adata_pixel_all.obs['cellidx'] = adata_pixel_all.obs['cellidx'].astype('category')

    cell_label = np.zeros_like(cellidx).astype('str')
    sc_label = np.array(adata.obs['Cluster'])
    for i in range(len(cellidx)):
        if cellidx[i]==0:
            continue

        cur_label = sc_label[int(cellidx[i]-1)]
        cell_label[i] = cur_label

    adata_pixel_all.obs['cell_label'] = cell_label.astype('str')
    adata_pixel_all.obs['cell_label'] = adata_pixel_all.obs['cell_label'].astype('category')

    adata_pixel_all.obsm['spatial'] = generate_spatialmat(n_pixels_side)

    sc.pl.embedding(adata_pixel_all,basis='spatial',color=['mask','cell_label'],size=10)

    adata_pixel_all.write_h5ad(save_file)
    return adata_pixel_all

In [ ]:
adata_list = []
for i in range(58):
    Pn = i+1
    print(Pn)
    cur_adata = scMEP_h5ad(Pn)
    adata_list.append(cur_adata)